In [1]:
%cd ../..

/data/projects/logproc_ws/logits-processor-zoo


In [2]:
from examples.vllm.utils import vLLMRunner
from logits_processor_zoo.vllm import MaxTimeLogitProcessor


runner = vLLMRunner("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

example_prompts = [
"""
A farmer has a rectangular field. The length of the field is 20 meters longer than its width. 
If the perimeter of the field is 200 meters, find the dimensions of the field.
""",
]

/data/envs/logproc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-03 13:42:07 [__init__.py:244] Automatically detected platform cuda.


2025-07-03 13:42:10,216	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-03 13:42:21 [config.py:823] This model supports multiple tasks: {'generate', 'reward', 'score', 'classify', 'embed'}. Defaulting to 'generate'.
WARNING 07-03 13:42:21 [config.py:3271] Casting torch.bfloat16 to torch.float16.
WARNING 07-03 13:42:21 [cuda.py:91] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-03 13:42:21 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.1) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  devi

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.58it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.57it/s]


INFO 07-03 13:42:25 [default_loader.py:272] Loading weights took 0.76 seconds


INFO 07-03 13:42:25 [model_runner.py:1203] Model loading took 3.3461 GiB and 1.694513 seconds
INFO 07-03 13:42:26 [worker.py:294] Memory profiling takes 0.53 seconds
INFO 07-03 13:42:26 [worker.py:294] the current vLLM instance can use total_gpu_memory (79.19GiB) x gpu_memory_utilization (0.90) = 71.27GiB
INFO 07-03 13:42:26 [worker.py:294] model weights take 3.35GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 1.46GiB; the rest of the memory reserved for KV Cache is 66.31GiB.
INFO 07-03 13:42:26 [executor_base.py:113] # cuda blocks: 155208, # CPU blocks: 9362
INFO 07-03 13:42:26 [executor_base.py:118] Maximum concurrency for 16384 tokens per request: 151.57x
INFO 07-03 13:42:28 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 2.97 seconds


## Default Response

In [3]:
runner.generate_response(example_prompts, max_tokens=4096)

Prompt: 
A farmer has a rectangular field. The length of the field is 20 meters longer than its width. 
If the perimeter of the field is 200 meters, find the dimensions of the field.

First, I'll define the width of the field as \( w \) meters. Since the length is 20 meters longer than the width, the length will be \( w + 20 \) meters.

Next, I'll use the formula for the perimeter of a rectangle, which is \( 2 \times (\text{length} + \text{width}) \). Plugging in the expressions for length and width, the equation becomes:
\[
2(w + (w + 20)) = 200
\]

Simplifying the equation:
\[
2(2w + 20) = 200 \\
4w + 40 = 200 \\
4w = 160 \\
w = 40
\]

Finally, the width is 40 meters, and the length is \( 40 + 20 = 60 \) meters.
</think>

Let's solve the problem step by step.

**Given:**
- The field is rectangular.
- The length is \( 20 \) meters longer than the width.
- The perimeter of the field is \( 200 \) meters.

**Let:**
- \( w \) = width of the field (in meters)
- \( l \) = length of the fiel

## Interrupt after N seconds

In [4]:
runner.generate_response(example_prompts,
                         [MaxTimeLogitProcessor(runner.tokenizer, 
                                                boost_factor=0.1, 
                                                p=2, 
                                                complete_sentences=False, 
                                                max_time=1,
                                                )],
                        max_tokens=2000)

Prompt: 
A farmer has a rectangular field. The length of the field is 20 meters longer than its width. 
If the perimeter of the field is 200 meters, find the dimensions of the field.

First, I'll define the width of the field as \( w \) meters. Since the length is 20 meters longer than the width, the length will be \( w + 20 \) meters.

Next, I'll use the formula for the perimeter of a rectangle, which is \( 2 \times (\text{length} + \text{width}) \). Plugging in the expressions for length and width, the equation becomes:
\[
2(w + (w + 20)) = 200
\]


-----END-----



## Combine with TriggerPhrase 

In [5]:
from logits_processor_zoo.vllm import TriggerPhraseLogitsProcessor

hurry_up = TriggerPhraseLogitsProcessor(runner.tokenizer,
                                        phrase="<Interruption> The time is over. The final answer is:", 
                                        trigger_time=2.5, 
                                        trigger_count=1, 
                                        trigger_after=True)
max_time = MaxTimeLogitProcessor(runner.tokenizer,
                                 boost_factor=0.1,
                                 p=2,
                                 complete_sentences=True,
                                 max_time=3)

runner.generate_response(example_prompts,
                         [hurry_up, max_time],
                        max_tokens=4096)

Prompt: 
A farmer has a rectangular field. The length of the field is 20 meters longer than its width. 
If the perimeter of the field is 200 meters, find the dimensions of the field.

First, I'll define the width of the field as \( w \) meters. Since the length is 20 meters longer than the width, the length will be \( w + 20 \) meters.

Next, I'll use the formula for the perimeter of a rectangle, which is \( 2 \times (\text{length} + \text{width}) \). Plugging in the expressions for length and width, the equation becomes:
\[
2(w + (w + 20)) = 200
\]

Simplifying the equation:
\[
2(2w + 20) = 200 \\
4w + 40 = 200 \\
4w = 160 \\
w = 40
\]

Finally, the width is 40 meters, and the length is \( 40 + 20 = 60 \) meters.
</think>

Let's solve the problem step by step.

**Given:**
- The field is rectangular.
- The length is \( 20 \) meters longer than the width.
- The perimeter of the field is \( 200 \) meters.

**Let:**
- \( w \) = width of the field (in meters)
- \( l \) = length of the fiel